In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import tensorflow as tf
import tensorflow_hub as hub
from nltk import sent_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import math
import keras
import nltk
%matplotlib inline

In [2]:
#loading dataset
df = pd.read_csv("Text_Similarity_Dataset.csv")

In [3]:
df.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


In [4]:
#Cleaning text and removing unwanted words and symbols
import re
def train(text_list):
    clean = []
    for word in text_list:
        word = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-.:;<=>?@[\]^`{|}~"""), '',word)
        word = word.replace('[]','')
        word = re.sub('\d+',' ',word)
        word = word.lower()
        clean.append(word)
    return clean

In [6]:
#we need to use Sentence encoding as many sentences are similar in textual context
X=[]
with tf.compat.v1.Graph().as_default():
    #loading model
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    print("embed",embed)
    messages = tf.compat.v1.placeholder(dtype=tf.string, shape=[None])
    output = embed(messages)
    #tensorflow session
    with tf.compat.v1.Session() as session:
        session.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])
        for text in ["text1","text2"]:
            phrases_emb_list = []
            for phrases in tqdm_notebook(df[text]): 
                #breaking paragraph into sentences
                sent_list = sent_tokenize(phrases)
                #cleaning the sentence
                clean_sent_list = train(sent_list)
                #feeding sentence in the encoder
                sent_embed = session.run(output, feed_dict={messages: clean_sent_list})
                #appending the encodings in a list
                phrases_emb_list.append(sent_embed.mean(axis=0).reshape(1,512))
            X.append(phrases_emb_list)

embed <tensorflow_hub.module.Module object at 0x000001D981573C40>
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
<ipython-input-6-620f62055d8a>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for phrases in tqdm_notebook(df[text]):


In [7]:
# saving the encodings for future use
encodings = open("enc.pkl", "wb")
pickle.dump(X, encodings)
encodings.close()

In [8]:
# Converting the encoding into numpy arrays
text1 = np.array(X[0])
text2 = np.array(X[1])

In [9]:
#converting data to numpy aaray
X_data=np.array(X)

In [10]:
def get_cosine(vec1, vec2):
     
     numerator = sum(vec1 * vec2 )

     sum1 = sum([vec1[x]**2 for x in range(len(vec1))])
     sum2 = sum([vec2[x]**2 for x in range(len(vec2))])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

In [11]:
values = []
for i in range(len(df)):
    cosine=get_cosine(text1[i][0], text2[i][0])
    values.append(cosine)
df["similarity_score"] = values

In [12]:
df.head()

,Unique_ID,text1,text2,similarity_score
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...,0.166281
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...,0.640406
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...,0.737810
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...,0.693244
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...,0.636458
